In [1]:
!apt-get install python3-dev graphviz libgraphviz-dev pkg-config


zsh:1: command not found: apt-get


In [43]:
# https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/langgraph/00-langgraph-intro.ipynb

In [26]:
!pip install -qU langchain-openai==0.1.3 
!pip install -qU langchain-core==0.1.42
!pip install -qU langgraph==0.0.37
!pip install -qU langchainhub==0.1.15 


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.1.42 which is incompatible.


In [56]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain.agents import create_openai_tools_agent
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.agents import AgentFinish
from langchain_core.prompts.chat import MessagesPlaceholder,HumanMessagePromptTemplate
import operator
import os
import json
from langgraph.graph import StateGraph, END
from typing import Optional
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate


SYSTEM_PROMPT_AUGMENTED_JP = """
You are an advanced apartment search assistant. Your goal is to assist users in finding apartments based on their preferences. You have access to several tools that can help gather user preferences, check if all necessary preferences are provided, and search for apartments using a Dummy API. Use these tools as needed to complete the user's request.

Available tools:
1. ask_user: Use this tool to communicate information or ask questions to the user.
2. search: Use this tool to search for apartments based on the provided preferences. Ensure all preferences are gathered before calling this tool.

Note: If you use tools, make sure the response object format is correct.

Instructions:
- If any preference is missing, use tool:[ask_user] to ask the user for the missing information.
- Once all preferences are provided, use too:[search] to find suitable apartments.
- Ensure clear and polite communication with the user at all times.

For example, if the user asked like "I am looking for apartment", you can ask "Do you have any preferences for the apartment?" using the tool:[ask_user]. 
"""

class AgentState(TypedDict):
    input: str
    agent_out: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
graph = StateGraph(AgentState)

@tool("ask_user")
def statement_for_user(statement: str):
    """Send a statement to the user."""
    return statement

@tool("search")
def search_apartments(isCityView: Optional[bool] = None, schools: Optional[str] = None, bedsMax: Optional[int] = None, maxPrice: Optional[int] = None, location: Optional[str] = None):
    """Search for apartments based on user preferences."""
    params = {
        "isCityView": isCityView,
        "schools": schools,
        "bedsMax": bedsMax,
        "maxPrice": maxPrice,
        "location": location
    }
    filtered_params = {k: v for k, v in params.items() if v is not None}
    return 'dummy response'

@tool("final_answer")
def final_answer_tool(
    answer: str,
    source: str
):
    """Returns a natural language response to the user in `answer`, and a
    `source` which provides citations for where this information came from.
    """
    return ""

llm = ChatOpenAI(temperature=0)

primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",SYSTEM_PROMPT_AUGMENTED_JP),
        MessagesPlaceholder(variable_name='chat_history', optional=True), 
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), 
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)

query_agent_runnable = create_openai_tools_agent(
    llm=llm,
    tools=[statement_for_user,search_apartments],
    prompt=primary_assistant_prompt
)

def run_query_agent(state: list):
    print("> run_query_agent")
    agent_out = query_agent_runnable.invoke(state)
    return {"agent_out": agent_out}

def execute_ask_user_preferences(state: list):
    print("> run_ask_user_preferences")
    action = state["agent_out"]
    print(f"action: {action}")
    tool_call = action[-1].message_log[-1].additional_kwargs["tool_calls"][-1]
    print(f"tool_call: {tool_call}")
    out = json.loads(tool_call["function"]["arguments"])
    print(f"out: {out}")
    return {"intermediate_steps": [{"statement": str(out)}]}

def execute_search_apartments(state: list):
    print("> execute_search")
    action = state["agent_out"]
    tool_call = action[-1].message_log[-1].additional_kwargs["tool_calls"][-1]
    out = search_apartments.invoke(
        json.loads(tool_call["function"]["arguments"])
    )
    return {"intermediate_steps": [{"search": str(out)}]}

def router(state: list):
    print("> router")
    if isinstance(state["agent_out"], list):
        return state["agent_out"][-1].tool
    else:
        return "error"

# finally, we will have a single LLM call that MUST use the final_answer structure
final_answer_llm = llm.bind_tools([final_answer_tool], tool_choice="final_answer")

# # we use the same forced final_answer LLM call to handle incorrectly formatted
# # output from our query_agent
def handle_error(state: list):
    print("> handle_error")
    query = state["input"]
    prompt = f"""You are a helpful assistant, answer the user's question.

    QUESTION: {query}
    """
    out = final_answer_llm.invoke(prompt)
    function_call = out.additional_kwargs["tool_calls"][-1]["function"]["arguments"]
    return {"agent_out": function_call}

graph.add_node("query_agent", run_query_agent)
graph.add_node("search", execute_search_apartments)
graph.add_node("error", handle_error)
graph.add_node("ask_user", execute_ask_user_preferences)
graph.set_entry_point("query_agent")
graph.add_conditional_edges(
    start_key="query_agent",  # where in graph to start
    condition=router,  # function to determine which node is called
    conditional_edge_mapping={
        "search": "search",
        "ask_user": "ask_user",
        "error": "error",
        "final_answer": END
    }
)
graph.add_edge("error", END)
graph.add_edge("search", END)
graph.add_edge("ask_user", END)

runnable = graph.compile()


In [57]:
out = runnable.invoke({
    "input": "I am looking for apartment",
    "chat_history": []
})
print(out)
print(out["agent_out"])

> run_query_agent
> router
> run_ask_user_preferences
action: [ToolAgentAction(tool='ask_user', tool_input={'statement': 'Do you have any preferences for the apartment?'}, log="\nInvoking: `ask_user` with `{'statement': 'Do you have any preferences for the apartment?'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_J4z3FDbqNXtstKsuwchlrWf5', 'function': {'arguments': '{"statement":"Do you have any preferences for the apartment?"}', 'name': 'ask_user'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 311, 'total_tokens': 333}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2b2d3c5e-17ac-4a68-afbf-411aeb3bdeb8-0', tool_calls=[{'name': 'ask_user', 'args': {'statement': 'Do you have any preferences for the apartment?'}, 'id': 'call_J4z3FDbqNXtstKsuwchlrWf5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 311, 'out